In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from loguru import logger
from pathlib import Path
from scipy import signal

import shap
from sklearn.metrics import classification_report
from scipy import signal
import keras
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Activation, Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf

/home/aseliverstov/.cache/pypoetry/virtualenvs/brain-signals-_5HxkjSc-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-14 01:33:26.849307: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-14 01:33:26.919485: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-14 01:33:27.003864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736807607.081099  129639 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00

In [2]:
class EEG:
    def __init__(self, data_dir: Path, intencities: list[float], n_channels: int):
        self.data_dir = data_dir
        self.intencities = intencities
        self.n_channels = n_channels

    def load_participant(self, participant_id: int):
        self.int = {}
        self.delta = {}
        self.theta = {}
        self.alpha = {}
        self.beta = {}
        self.gamma = {}

        for i in self.intencities:
            self.int[i] = np.loadtxt(self.data_dir / f"Participant {participant_id}" / "Figs for spectra" / f"Backgr_int_{i}.dat")
            delta_signal = np.empty(self.int[i].shape)
            theta_signal =  np.empty(self.int[i].shape)
            alpha_signal = np.empty(self.int[i].shape)
            beta_signal = np.empty(self.int[i].shape)
            gamma_signal = np.empty(self.int[i].shape)
            for c in range(self.int[i].shape[1]):
                delta_signal[:, c], theta_signal[:, c], alpha_signal[:, c], beta_signal[:, c], gamma_signal[:, c] = self._fir_filtering(i, c)

            self.delta[i] = delta_signal
            self.theta[i] = theta_signal
            self.alpha[i] = alpha_signal
            self.beta[i] = beta_signal 
            self.gamma[i] = gamma_signal

        participant_info = {}
        participant_info["delta"] = self.delta
        participant_info["theta"] = self.theta
        participant_info["alpha"] = self.alpha
        participant_info["beta"] = self.beta
        participant_info["gamma"] = self.gamma

        return participant_info

    def _fir_filtering(self, i, c):
        filter_delta = signal.firwin(400, [1.0, 4.0], pass_zero=False, fs=250)
        filter_theta = signal.firwin(400, [5.0, 8.0], pass_zero=False, fs=250)
        filter_alpha = signal.firwin(400, [8.0, 12.0], pass_zero=False, fs=250)
        filter_beta = signal.firwin(400, [13.0, 30.0], pass_zero=False, fs=250)
        filter_gamma = signal.firwin(400, [31.0, 45.0], pass_zero=False, fs=250)

        res_delta = signal.convolve(self.int[i][:, c], filter_delta, mode='same')
        res_theta = signal.convolve(self.int[i][:, c], filter_theta, mode='same')
        res_alpha = signal.convolve(self.int[i][:, c], filter_alpha, mode='same')
        res_beta = signal.convolve(self.int[i][:, c], filter_beta, mode='same')
        res_gamma = signal.convolve(self.int[i][:, c], filter_gamma, mode='same')

        return res_delta, res_theta, res_alpha, res_beta, res_gamma

def process_signal_type_participant(dataset: EEG, user: int, signal_type: str):
    eeg_person = dataset.load_participant(user)

    person_signals = []
    signal_index = []
    for i in dataset.intencities:
        for ch in range(dataset.n_channels):
            signal_series = eeg_person[signal_type].get(i)[:, ch][:15000]
            person_signals.append(signal_series)

            signal_index.append([user, i, ch])            

    person_signal_index_df = pd.DataFrame(signal_index, columns=["user", "intensity", "channel"])
    person_signal_index_df["is_left_channel"] = np.where(person_signal_index_df["channel"] % 2 == 0, 1, 0)
    person_signal_index_df["signal_type"] = signal_type

    person_signals_df = pd.DataFrame(person_signals)

    return pd.concat([person_signal_index_df, person_signals_df], axis=1)

In [3]:
data_dir = Path("./dataset")
intencities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
n_channels = 31

dataset = EEG(data_dir=data_dir, intencities=intencities, n_channels=n_channels)

In [4]:
data = []

for user in [1, 2, 3, 4, 5]:
    for signal_type in ["delta", "theta", "alpha", "beta", "gamma"]:
        data.append(process_signal_type_participant(dataset, user, signal_type))

data = pd.concat(data, axis=0).reset_index(drop=True)

data.head()

,user,intensity,channel,is_left_channel,signal_type,0,1,2,3,4,...,14990,14991,14992,14993,14994,14995,14996,14997,14998,14999
0,1,0.1,0,1,delta,32.819580,22.394499,11.944530,1.509794,-8.869929,...,5.761321,8.584314,11.482307,14.444269,17.451520,20.490840,23.550929,26.612040,29.656919,32.678659
1,1,0.1,1,0,delta,22.895093,22.571077,22.213273,21.817371,21.388242,...,-36.569596,-37.383749,-38.258960,-39.193345,-40.186622,-41.226317,-42.301337,-43.410787,-44.548579,-45.696425
2,1,0.1,2,1,delta,22.636168,25.515590,28.336306,31.081621,33.736376,...,-57.224056,-57.872630,-58.465101,-58.998834,-59.471828,-59.881532,-60.224889,-60.496918,-60.690027,-60.795766
3,1,0.1,3,0,delta,36.106648,38.336931,40.448344,42.426091,44.260684,...,-9.807186,-7.906850,-5.966429,-3.996789,-2.011164,-0.023158,1.954782,3.909010,5.827176,7.701683
4,1,0.1,4,1,delta,26.015135,23.859016,21.597520,19.234030,16.773968,...,-20.737286,-19.244307,-17.681272,-16.058450,-14.386304,-12.677365,-10.941320,-9.185652,-7.417999,-5.644647


# Replication

In [5]:
# data = data[(data["signal_type"] == "delta") & (data["user"] == 1)]

X = data.iloc[:, 5:].to_numpy()
y = data["is_left_channel"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model = Sequential()
model.add(Dense(2500, input_shape= (X_train.shape[1],), activation='tanh'))
model.add(Dense(1000, activation='tanh'))
model.add(Dense(500, activation='tanh'))
model.add(Dense(200, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(25, activation='tanh'))
model.add(Dense(15, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=2)
y_pred = model.predict(X_test).tolist()

classification_report(y_test, np.round(y_pred))

/home/aseliverstov/.cache/pypoetry/virtualenvs/brain-signals-_5HxkjSc-py3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-14 01:34:40.362610: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2500)           │    37,502,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1000)           │     2,501,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 500)            │       500,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 200)            │       100,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,631,031 (155.00 MB)

 Trainable params: 40,631,031 (155.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
25/25 - 10s - 414ms/step - accuracy: 0.4758 - loss: 0.7358
Epoch 2/20
25/25 - 9s - 359ms/step - accuracy: 0.5524 - loss: 0.6969
Epoch 3/20
25/25 - 9s - 348ms/step - accuracy: 0.5081 - loss: 0.7182
Epoch 4/20
25/25 - 9s - 346ms/step - accuracy: 0.5282 - loss: 0.6909
Epoch 5/20
25/25 - 9s - 349ms/step - accuracy: 0.5605 - loss: 0.6885
Epoch 6/20
25/25 - 9s - 352ms/step - accuracy: 0.5565 - loss: 0.6870
Epoch 7/20
25/25 - 9s - 346ms/step - accuracy: 0.5766 - loss: 0.6664
Epoch 8/20
25/25 - 9s - 348ms/step - accuracy: 0.5403 - loss: 0.6814
Epoch 9/20
25/25 - 9s - 348ms/step - accuracy: 0.5040 - loss: 0.6914
Epoch 10/20
25/25 - 9s - 344ms/step - accuracy: 0.5282 - loss: 0.6825
Epoch 11/20
25/25 - 9s - 345ms/step - accuracy: 0.5524 - loss: 0.6721
Epoch 12/20
25/25 - 9s - 347ms/step - accuracy: 0.5847 - loss: 0.6590
Epoch 13/20
25/25 - 8s - 339ms/step - accuracy: 0.6169 - loss: 0.6385
Epoch 14/20
25/25 - 8s - 338ms/step - accuracy: 0.5444 - loss: 0.6695
Epoch 15/20
25/25 - 8s - 337

AttributeError: 'Sequential' object has no attribute 'predict_classes'